In [ ]:
%cd ..
%load_ext autoreload
%autoreload 2

RTS_DATA = "/mnt/d/rts/"
METADATA = RTS_DATA + 'metadata'
VIDEOS = RTS_DATA + '0'

In [53]:
import torch
torch.cuda.is_available()

True

In [45]:
from datetime import datetime
import meilisearch
import orjson
import pandas as pd
import zipfile
import os
from typing import Dict, List, Optional, Tuple

In [14]:
client = meilisearch.Client('http://localhost:7700', '1234')
client.get_keys()
 
# int(datetime.now().timestamp())
# client.create_index('books', {'primaryKey': 'reference_number'})

{'results': [{'name': 'Default Search API Key',
   'description': 'Use it to search from the frontend',
   'key': 'cf2fed9b0f2a363c008b02c850f0dfac26d87565808a17f6db5f2b4edc0f8b8d',
   'uid': '82d3a743-9dc2-44db-80a5-6f79a1833413',
   'actions': ['search'],
   'indexes': ['*'],
   'expiresAt': None,
   'createdAt': '2022-12-06T10:48:39.670505368Z',
   'updatedAt': '2022-12-06T10:48:39.670505368Z'},
  {'name': 'Default Admin API Key',
   'description': 'Use it for anything that is not a search operation. Caution! Do not expose it on a public frontend',
   'key': '9578bcc35e0f5ad675ff935647e1f4b83b96ec5e33dd39c5332a8bbdd72e0826',
   'uid': '7c6ce404-1fd4-416d-a73d-c6453abb77f7',
   'actions': ['*'],
   'indexes': ['*'],
   'expiresAt': None,
   'createdAt': '2022-12-06T10:48:39.66668103Z',
   'updatedAt': '2022-12-06T10:48:39.66668103Z'}],
 'offset': 0,
 'limit': 20,
 'total': 2}

In [49]:
def parse_item_sequences(data: Dict, media: Dict) -> Dict:
    pass

def parse_item(raw: Dict) -> List[Dict]:
    def find_media_id(support: Dict) -> Tuple[Optional[int], Optional[str]]:
        for i, d in enumerate(support):
            if d.startswith('Z'):
                return i, d
        return None, None

    d = raw['response']['docs'][0]
    seq_idx, media_id = find_media_id(d['idSupport'])
    duration = d['DureeMediaSec'][seq_idx]
    ratio = d['RatioMedia'][seq_idx]

    media = {
        'guid': d['Guid'],
        'mediaId': media_id,
        'mediaDuration': duration,
        'ratio': ratio,
        'formatType': d['FormatMedia'][seq_idx],
        'formatResolution': d['DefinitionMedia'][seq_idx],
        'materialType': d['TypeMaterielMedia'][seq_idx],
        'publishedDate': d.get('DatePublication'),
        'publishedBy': d.get('CanalPublication'),

        'rights': d.get('SemaphoreDroit'),
        'categoryName': d.get('CategorieAsset'),
        'assetType': d.get('TypeAsset'),
        'contentType': d.get('TypeContenu'),
        'backgoundType': d.get('idTypeBackground'),
        
        'title': d.get('Titre'),
        'resume': d.get('Resume'),
        'resumeSequence': d.get('ResumeSequence'),
        'geoTheme': d.get('ThematiquesGEO'),

    }

    seqs = parse_item_sequences(d, media)
    return seqs
    

def read_metadata_zippart(root_dir: str, part_num: int) -> pd.DataFrame:
    path = os.path.join(root_dir, f'{part_num}.zip')
    with zipfile.ZipFile(path, 'r') as z:
        jsons = zipfile.Path(z, f'{part_num}/jsonData/')
        srts = zipfile.Path(z, f'{part_num}/s2txt/')

        for j in jsons.iterdir():
            with j.open('r') as fp:
                js = orjson.loads(fp.read())
                clean = parse_item(js)
                STOP
            # STOP

# read_metadata_zippart(METADATA, 0)

In [52]:
with open('raw/AA1101000650_0.txt', 'r', encoding='latin-1') as fp:
    text = fp.read()
    print(text)

UMID : ZE003073
Date de transcription : 27-DEC-18
Nom du modèle : fre-daily

Locuteur MS4 (0:00:00 > 0:00:01) [78%] : 
Maman bonjour.


Locuteur FS1 (0:00:01 > 0:00:23) [87%] : 
Moi j'aime bien votre émission mais, et c'est un gros mais votre musique de générique est épouvantable : hausse pour c'est stressant et agressif Monsieur Tornay votre tenue vestimentaire terre est irrespectueuse face à un conseiller fédéral et quel drôle de fond rose, serait-ce pour mieux faire passer la hausse des primes, c'est l'augmentation des primes qui nous rend malade, donc j'ai des scellés.


Locuteur FS2 (0:00:39 > 0:00:40) [99%] : 
Luc.


Locuteur FS4 (0:00:50 > 0:01:01) [92%] : 
Bonsoir, bienvenue, merci d'être avec nous pour cette nouvelle émission de mise au point, révélation, émotion, évasion, ce sont les ingrédients du programme de ce soir, on découvrent tout de suite en images.


Locuteur FS1 (0:01:02 > 0:01:34) [88%] : 
Affaire Hainard : 2ème : épisode révélation confession, l'inspecteur HEIMO 

In [56]:
import whisper

model = whisper.load_model("base")
# result = model.transcribe("audio.mp3")
# print(result["text"])

/home/kikohs/.cache/pypoetry/virtualenvs/rts-DIiCth0j-py3.8/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
100%|███████████████████████████████████████| 139M/139M [00:03<00:00, 37.5MiB/s]
/home/kikohs/.cache/pypoetry/virtualenvs/rts-DIiCth0j-py3.8/lib/python3.8/site-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
